Load packages

In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pe

import warnings
warnings.filterwarnings("ignore")

from sklearn import preprocessing
from sklearn_extra.cluster import KMedoids
from collections import Counter
from floris import FlorisModel
from floris import TimeSeries
from floris.optimization.yaw_optimization.yaw_optimizer_geometric import YawOptimizationGeometric

# import distributions
from scipy.stats import dweibull, norm, vonmises

# count computing time
from time import perf_counter as timerpc
start_time = timerpc() # start counting time

Create WF

In [2]:
# Load an input file
fmodel = FlorisModel("cc.yaml") # CumulativeCurl model

# Get WTs coordinates
coord = pd.read_excel('WF_coordinates.xlsx')
x = coord.x
y = coord.y

# initialize FLORIS
fmodel.set(layout_x=x, layout_y=y, reference_wind_height=90)

Load data

In [3]:
data = pd.read_excel('da_data_11.xlsx') # import wind conditions and da price data
data_fr = pd.read_excel('fr_data.xlsx', header = None) # import FR data
t_data = np.flip(data_fr.iloc[:, 0].to_numpy())
CDF = np.flip(data_fr.iloc[:, 1].to_numpy())
results_power_curve = np.zeros([24, 5]) # array of results: row = hour, c0 = Pmax, c1 = Pe, c2 = Pmfr, c3 = Pfr, c4 = income
results_baseline = np.zeros([24, 5])
results_wrc1 = np.zeros([24, 5]) # results for wrc strategy 1
results_wrc2 = np.zeros([24, 4]) # results for wrc strategy 2 (c0 = Pe, c1 = Pmfr, c2 = Pfr, c3 = income)
real_power_curve = np.zeros([24, 1]) # estimated real income for Power Curve

Constants

In [4]:
# scenarios
n = 1000 # number of random variables to generate
n_clusters = 15 # n° of clusters of scenarios

# reserve prices
lambda_up_mfr = 2.5 # holding payment of upward reserve [£/MW/h]
lambda_a_fr = 3.48 # Holding payment of FR reserve [£/MW/h]
lambda_e_fr = 87.25 # Price of FR energy deployment [£/MWh]
lambda_delta_fr = 1.2*lambda_e_fr # Penalty for FR imbalance [£/MWh]

# WF parameters
y_min = -25 # minimum yaw angle [°]
y_max = 25 # maximum yaw angle [°]
Pn = 630 # nominal output power of WF [MW]
Pmin = 0 # minimum output power for WF [MW]
Pfr_min = 25 # minimum FR power requirement [MW]

Optimisation framework

In [5]:
for h in range(24): # h = hour of the day

    # DATA
    ## Data for hour h
    ws_avg = data.loc[h].Ws_avg # average wind speed for hour h
    ws_std = data.loc[h].Ws_std # standard deviation of wind speed for hour h
    wd_avg = data.loc[h].Wd_avg # average wind direction for hour h
    wd_std = data.loc[h].Wd_std # standard deviation of wind direction for hour h
    ti = data.loc[h].TI # turbulence intensity for hour h
    price_da = data.loc[h].Price_DA # day-ahead spot price for hour h

    # SCENARIOS
    ### von Mises parameters
    circ_mean = np.deg2rad(wd_avg) # circular mean [rad]
    kappa = 1/np.deg2rad(wd_std)**2
    
    ## Scenario Generation
    ### generate random variables
    f_rand = dweibull.rvs(1.5, loc=50, scale=0.0632, size=n) # random frequency variables
    ws_rand = norm.rvs(loc=ws_avg, scale=ws_std, size=n) # random wind speed variables
    wd_rand = np.rad2deg(vonmises.rvs(kappa, loc=circ_mean, size=n)) # random wind direction variables
    t1 = np.zeros(n ,dtype=int) # FR duration for first 30min
    t2 = np.zeros(n ,dtype=int) # FR duration for second 30min
    for k in np.arange(n):
        a_1 = np.random.uniform() # sample from uniform distribution
        a_2 = np.random.uniform() # sample from uniform distribution
        t1[k] = t_data[np.min(np.where(CDF>=a_1)[0][0])-1]  # location where random number is first time larger than the CDF, rounded to closest element.
        t2[k] = t_data[np.min(np.where(CDF>=a_2)[0][0])-1] # location where random number is first time larger than the CDF, rounded to closest element.
    t1_rand = (60-t1)/60 # convert to hours
    t2_rand = (60-t2)/60 # convert to hours
    t_rand = t1_rand + t2_rand # FR duration scenarios for hour h
    
    ### normalize vectors so each variable has the same weight
    f_n, n_f = preprocessing.normalize([f_rand], axis=1, copy=False, return_norm=True) # normalize frequency
    ws_n, n_ws = preprocessing.normalize([ws_rand], axis=1, copy=False, return_norm=True) # normalize wind speed
    wd_n, n_wd = preprocessing.normalize([wd_rand], axis=1, copy=False, return_norm=True) # normalize wind direction
    t_n, n_t = preprocessing.normalize([t_rand], axis=1, copy=False, return_norm=True) # normalize FR duration
    
    ### create arrays
    var_rand = np.column_stack((f_n.reshape(n, 1), ws_n.reshape(n, 1), wd_n.reshape(n, 1), t_n.reshape(n, 1))) # array of all normalized variables
    norms = np.array([n_f, n_ws, n_wd, n_t]).reshape(1, 4) # array of norms

    ## K-medoids Clustering
    ###  clustering
    kmedoids = KMedoids(n_clusters=n_clusters).fit(var_rand) 
    medoids = kmedoids.cluster_centers_*norms
    
    ### get probability of each scenario
    numbers = Counter(kmedoids.labels_) # n° of variables in each cluster
    probabilities = np.array([]) # create array of probabilities
    for i in range(n_clusters): 
        p_i = numbers[i]/n # probability of  scenario_i = n° of variables in cluster_i/n
        probabilities = np.append(probabilities,[p_i],axis=0) # store probability of scenario_i
    probabilities = probabilities.reshape(n_clusters,1) # reshape array
    
    ### scenarios for optimisation
    scenarios = np.append(medoids, probabilities, axis=1) # row = scenario n°, c0 = f, c1 = ws, c2 = wd, c3 = t , c4 = oprobability of scenario

    # PARAMETERS
    ## Prices
    lambda_e = price_da # price of energy [£/MWh]
    lambda_delta_e = 1.2*lambda_e # Penalty for energy imbalance [£/MWh]

    ## Pmax
    fmodel.reset_operation() # reset yaw angles
    time_series_pmax = TimeSeries(wind_speeds=np.array([ws_avg]), wind_directions=wd_avg, turbulence_intensities=ti) # initialize wind conditions
    fmodel.set(wind_data=time_series_pmax)
    fmodel.run_no_wake()
    Pmax_power_curve = fmodel.get_farm_power()[0]/1E6 # maximum WF output power for Power Curve [MW]
    fmodel.run()
    Pmax_baseline = fmodel.get_farm_power()[0]/1E6 # maximum WF output power for Baseline [MW]
    opt = YawOptimizationGeometric(fmodel, minimum_yaw_angle=y_min, maximum_yaw_angle=y_max).optimize()
    yaw_angles = np.vstack(opt.yaw_angles_opt)
    fmodel.set(yaw_angles=yaw_angles)
    fmodel.run()
    Pmax_wrc = fmodel.get_farm_power().squeeze()/1E6 # maximum WF output power for WRC [MW]

    ## Scenarios
    S = range(n_clusters) # number of scenarios
    p = scenarios[:, 4] # probabilities of scenarios
    t = scenarios[:, 3] # FR duration of scenarios

    ## Pmax(s)
    fmodel.reset_operation() # reset yaw angles
    time_series_pmax_s = TimeSeries(wind_speeds=scenarios[:, 1], wind_directions=scenarios[:, 2], turbulence_intensities=ti*np.ones(n_clusters)) # initialize wind conditions
    fmodel.set(wind_data=time_series_pmax)
    fmodel.run_no_wake()
    Pmax_s_power_curve = fmodel.get_farm_power()/1E6 # maximum WF output power for Power Curve of scenario s [MW]
    fmodel.run()
    Pmax_s_baseline = fmodel.get_farm_power()/1E6 # maximum WF output power for Baseline of scenario s [MW]
    opt = YawOptimizationGeometric(fmodel, minimum_yaw_angle=y_min, maximum_yaw_angle=y_max).optimize()
    yaw_angles = np.vstack(opt.yaw_angles_opt)
    fmodel.set(yaw_angles=yaw_angles)
    fmodel.run()
    Pmax_s_wrc = fmodel.get_farm_power().squeeze()/1E6 # maximum WF output power for WRC of scenario s [MW]

    # POWER CURVE
    model_power_curve = pe.ConcreteModel() # create power curve model object
    
    ## Power curve decision variables
    model_power_curve.Pe = pe.Var(within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Energy dispatch [MW]
    model_power_curve.Pmfr = pe.Var(within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Upward MFR reserve dispatch [MW]
    model_power_curve.Pfr = pe.Var(within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Upward FR power dispatch [MW]
    model_power_curve.Delta_Pfr = pe.Var(S, within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # FR power redispatch of scenario s [MW]
    model_power_curve.Delta_pe = pe.Var(S, within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Energy redispatchh of scenario s [MW]
    
    ## Power curve objective function
    model_power_curve.e = pe.Expression(expr = model_power_curve.Pe*lambda_e + model_power_curve.Pmfr*lambda_up_mfr + model_power_curve.Pfr*lambda_a_fr
                           + sum(p[s]*(model_power_curve.Pfr*t[s]*lambda_e_fr - ((((model_power_curve.Pfr - model_power_curve.Delta_Pfr[s])*t[s])**2)*lambda_delta_fr**2) - 
                                           (((model_power_curve.Pe - model_power_curve.Delta_Pe[s])**2)*lambda_delta_e**2)
                                           )for s in S))
    model_power_curve.obj = pe.Objective(expr = model_power_curve.e, sense=pe.maximize)
    
    ## Power curve constraints
    model_power_curve.c1 = pe.Constraint(expr = model_power_curve.Pe + model_power_curve.Pmfr + model_power_curve.Pfr <= Pmax_power_curve)
    model_power_curve.c2 = pe.Constraint(expr = model_power_curve.Pe >= Pmin)
    model_power_curve.c3 = pe.Constraint(expr = model_power_curve.Pmfr <= 0.1*model_power_curve.Pe) # Max 10% deloading
    model_power_curve.c4 = pe.Constraint(expr = model_power_curve.Pfr >= Pfr_min) # Pfr >= 25 MW
    def power_curve_rule(m, s):
        return  m.Delta_Pe[s] + m.Pmfr - m.Delta_Pfr[s] <= Pmax_s_power_curve[s]
    model_power_curve.c5 = pe.Constraint(S, rule=power_curve_rule)
    
    ## Solve Power curve
    pe.SolverFactory('ipopt').solve(model_power_curve, tee=False)

    # BASELINE
    model_baseline = pe.ConcreteModel() # create baseline model object
    
    ## Baseline decision variables
    model_baseline.Pe = pe.Var(within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Energy dispatch [MW]
    model_baseline.Pmfr = pe.Var(within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Upward MFR reserve dispatch [MW]
    model_baseline.Pfr = pe.Var(within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Upward FR power dispatch [MW]
    model_baseline.Delta_Pfr = pe.Var(S, within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # FR power redispatch of scenario s [MW]
    model_baseline.Delta_Pe = pe.Var(S, within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Energy redispatchh of scenario s [MW]
    
    ## Baseline objective function
    model_baseline.e = pe.Expression(expr = model_baseline.Pe*lambda_e + model_baseline.Pmfr*lambda_up_mfr + model_baseline.Pfr*lambda_a_fr
                           + sum(p[s]*(model_baseline.Pfr*t[s]*lambda_e_fr - ((((model_baseline.Pfr - model_baseline.Delta_Pfr[s])*t[s])**2)*lambda_delta_fr**2) - 
                                           (((model_baseline.Pe - model_baseline.Delta_Pe[s])**2)*lambda_delta_e**2)
                                           )for s in S))
    model_baseline.obj = pe.Objective(expr = model_baseline.e, sense=pe.maximize)
    
    ## Baseline constraints
    model_baseline.c1 = pe.Constraint(expr = model_baseline.Pe + model_baseline.Pmfr + model_baseline.Pfr <= Pmax_baseline)
    model_baseline.c2 = pe.Constraint(expr = model_baseline.Pe >= Pmin)
    model_baseline.c3 = pe.Constraint(expr = model_baseline.Pmfr <= 0.1*model_baseline.Pe) # Max 10% deloading
    model_baseline.c4 = pe.Constraint(expr = model_baseline.Pfr >= Pfr_min) # Pfr >= 25 MW
    def baseline_rule(m, s):
        return  m.Delta_Pe[s] + m.Pmfr - m.Delta_Pfr[s] <= Pmax_s_baseline[s]
    model_baseline.c5 = pe.Constraint(S, rule=baseline_rule)
    
    ## Solve Baseline
    pe.SolverFactory('ipopt').solve(model_baseline, tee=False)

    # WRC 1
    model_wrc1 = pe.ConcreteModel() # create WRC 1 model object
    
    ## WRC 1 decision variables
    model_wrc1.Pe = pe.Var(within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Energy dispatch [MW]
    model_wrc1.Pmfr = pe.Var(within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Upward MFR reserve dispatch [MW]
    model_wrc1.Pfr = pe.Var(within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Upward FR power dispatch [MW]
    model_wrc1.Delta_Pfr = pe.Var(S, within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # FR power redispatch of scenario s [MW]
    model_wrc1.Delta_Pe = pe.Var(S, within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Energy redispatchh of scenario s [MW]
    
    ## WRC 1 objective function
    model_wrc1.e = pe.Expression(expr = model_wrc1.Pe*lambda_e + model_wrc1.Pmfr*lambda_up_mfr + model_wrc1.Pfr*lambda_a_fr
                           + sum(p[s]*(model_wrc1.Pfr*t[s]*lambda_e_fr - ((((model_wrc1.Pfr - model_wrc1.Delta_Pfr[s])*t[s])**2)*lambda_delta_fr**2) - 
                                           (((model_wrc1.Pe - model_wrc1.Delta_Pe[s])**2)*lambda_delta_e**2)
                                           )for s in S))
    model_wrc1.obj = pe.Objective(expr = model_wrc1.e, sense=pe.maximize)
    
    ## WRC 1 constraints
    model_wrc1.c1 = pe.Constraint(expr = model_wrc1.Pe + model_wrc1.Pmfr + model_wrc1.Pfr <= Pmax_wrc)
    model_wrc1.c2 = pe.Constraint(expr = model_wrc1.Pe >= Pmin)
    model_wrc1.c3 = pe.Constraint(expr = model_wrc1.Pmfr <= 0.1*model_wrc1.Pe) # Max 10% deloading
    model_wrc1.c4 = pe.Constraint(expr = model_wrc1.Pfr >= Pfr_min) # Pfr >= 25 MW
    def wrc1_rule(m, s):
        return  m.Delta_Pe[s] + m.Pmfr - m.Delta_Pfr[s] <= Pmax_s_wrc[s]
    model_wrc1.c5 = pe.Constraint(S, rule=wrc1_rule)
    
    ## Solve WRC 1
    pe.SolverFactory('ipopt').solve(model_wrc1, tee=False)
    
    # WRC 2
    model_wrc2 = pe.ConcreteModel() # create WRC 2 model object
    
    ## WRC 2 decision variables
    model_wrc2.Pe = pe.Var(within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Energy dispatch [MWh]
    model_wrc2.Pmfr = pe.Var(within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Upward MFR reserve dispatch [MW]
    model_wrc2.Pfr = pe.Var(within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Upward FR power dispatch [MW]
    model_wrc2.Delta_Pfr = pe.Var(S, within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # FR power redispatch of scenario s [MW]
    model_wrc2.Delta_Pe = pe.Var(S, within = pe.NonNegativeReals, bounds = (0, None), initialize = 0) # Energy redispatchh of scenario s [MW]
    
    ## WRC 2 objective function
    model_wrc2.e = pe.Expression(expr = model_wrc2.Pe*lambda_e + model_wrc2.Pmfr*lambda_up_mfr + model_wrc2.Pfr*lambda_a_fr
                           + sum(p[s]*(model_wrc2.Pfr*t[s]*lambda_e_fr - ((((model_wrc2.Pfr - model_wrc2.Delta_Pfr[s])*t[s])**2)*lambda_delta_fr**2) - 
                                           (((model_wrc2.Pe - model_wrc2.Delta_Pe[s])**2)*lambda_delta_e**2)
                                           )for s in S))
    model_wrc2.obj = pe.Objective(expr = model_wrc2.e, sense=pe.maximize)
    
    ## WRC 2 constraints
    model_wrc2.c1 = pe.Constraint(expr = model_wrc2.Pe + model_wrc2.Pmfr + model_wrc2.Pfr <= Pmax_wrc)
    model_wrc2.c2 = pe.Constraint(expr = model_wrc2.Pe <= Pmax_baseline)
    model_wrc2.c3 = pe.Constraint(expr = model_wrc2.Pe >= Pmin)
    model_wrc2.c4 = pe.Constraint(expr = model_wrc2.Pmfr <= 0.1*model_wrc2.Pe) # Max 10% deloading
    model_wrc2.c5 = pe.Constraint(expr = model_wrc2.Pfr >= Pfr_min) # Pfr >= 25 MW
    def wrc2_rule(m, s):
        return  m.Delta_Pe[s] + m.Pmfr - m.Delta_Pfr[s] <= Pmax_s_wrc[s]
    model_wrc2.c6 = pe.Constraint(S, rule=wrc2_rule)
    
    ## Solve WRC 2
    pe.SolverFactory('ipopt').solve(model_wrc2, tee=False)

    # INCOME AND RESULTS
    ## Power curve
    Pe_power_curve = model_power_curve.Pe.value # Pe bid for power curve
    Pmfr_power_curve = model_power_curve.Pmfr.value # Pmfr bid for power curve
    Pfr_power_curve = model_power_curve.Pfr.value # Pfr bid for power curve
    income_power_curve = Pe_power_curve*lambda_e + Pmfr_power_curve*lambda_up_mfr + Pfr_power_curve*lambda_a_fr + \
    sum(p[s]*(Pfr_power_curve*t[s]*lambda_e_fr - abs((Pfr_power_curve - model_power_curve.Delta_Pfr[s].value)*t[s])*lambda_delta_fr 
             - abs((Pe_power_curve - model_power_curve.Delta_Pe[s].value)*lambda_delta_e))for s in S) # power curve expected income
    power_curve_hour = np.array([Pmax_power_curve, Pe_power_curve, Pmfr_power_curve, Pfr_power_curve, income_power_curve]) # power curve results for hour h
    results_power_curve[h] = power_curve_hour # store power curve results

    ## Basline
    Pe_baseline = model_baseline.Pe.value # Pe bid for baseline
    Pmfr_baseline = model_baseline.Pmfr.value # Pmfr bid for baseline
    Pfr_baseline = model_baseline.Pfr.value # Pfr bid for basline
    income_baseline = Pe_baseline*lambda_e + Pmfr_baseline*lambda_up_mfr + Pfr_baseline*lambda_a_fr + \
    sum(p[s]*(Pfr_baseline*t[s]*lambda_e_fr - abs((Pfr_baseline - model_baseline.Delta_Pfr[s].value)*t[s])*lambda_delta_fr 
             - abs((Pe_baseline - model_baseline.Delta_Pe[s].value)*lambda_delta_e))for s in S) # baseline expected income
    baseline_hour = np.array([Pmax_baseline, Pe_baseline, Pmfr_baseline, Pfr_baseline, income_baseline]) # baseline results for hour h
    results_baseline[h] = baseline_hour # store basline results

    ## WRC 1
    Pe_wrc1 = model_wrc1.Pe.value # Pe bid for WRC 1
    Pmfr_wrc1 = model_wrc1.Pmfr.value # Pmfr bid for WRC 1
    Pfr_wrc1 = model_wrc1.Pfr.value # Pfr bid for WRC 1
    income_wrc1 = Pe_wrc1*lambda_e + Pmfr_wrc1*lambda_up_mfr + Pfr_wrc1*lambda_a_fr + \
    sum(p[s]*(Pfr_wrc1*t[s]*lambda_e_fr - abs((Pfr_wrc1 - model_wrc1.Delta_Pfr[s].value)*t[s])*lambda_delta_fr 
             - abs((Pe_wrc1 - model_wrc1.Delta_Pe[s].value)*lambda_delta_e))for s in S) # WRC 1 expected income
    wrc1_hour = np.array([Pmax_wrc, Pe_wrc1, Pmfr_wrc1, Pfr_wrc1, income_wrc1]) # WRC 1 results for hour h
    results_wrc1[h] = wrc1_hour # store WRC 1 results

    ## WRC 2
    Pe_wrc2 = model_wrc2.Pe.value # Pe bid for WRC 2
    Pmfr_wrc2 = model_wrc2.Pmfr.value # Pmfr bid for WRC 2
    Pfr_wrc2 = model_wrc2.Pfr.value # Pfr bid for WRC 2
    income_wrc2 = Pe_wrc2*lambda_e + Pmfr_wrc2*lambda_up_mfr + Pfr_wrc2*lambda_a_fr + \
    sum(p[s]*(Pfr_wrc2*t[s]*lambda_e_fr - abs((Pfr_wrc2 - model_wrc2.Delta_Pfr[s].value)*t[s])*lambda_delta_fr 
             - abs((Pe_wrc2 - model_wrc2.Delta_Pe[s].value)*lambda_delta_e))for s in S) # WRC 2 expected income
    wrc2_hour = np.array([Pe_wrc2, Pmfr_wrc2, Pfr_wrc2, income_wrc2]) # WRC 2 results for hour h
    results_wrc2[h] = wrc2_hour # store WRC 2 results

    ## Real power curve
    real_income_power_curve = Pe_power_curve*lambda_e + Pmfr_power_curve*lambda_up_mfr + Pfr_power_curve*lambda_a_fr + \
    sum(p[s]*(Pfr_power_curve*t[s]*lambda_e_fr - abs((Pfr_power_curve - model_baseline.Delta_Pfr[s].value)*t[s])*lambda_delta_fr 
             - abs((Pe_power_curve - model_baseline.Delta_Pe[s].value)*lambda_delta_e))for s in S) # real power curve expected income
    real_power_curve[h] = real_income_power_curve # store real power curve results

Results

In [6]:
# Power curve results
power_curve = pd.DataFrame({'Pfmax': results_power_curve[:, 0], 'Pe': results_power_curve[:, 1], 'Pmfr': results_power_curve[:, 2],
                            'Pfr': results_power_curve[:, 3], 'Income': results_power_curve[:, 4]}) # Creating pandas dataframe from numpy array
power_curve.to_csv(r"Power_curve.csv", index=False) # export power curve results

# Baseline results
baseline = pd.DataFrame({'Pfmax': results_baseline[:, 0], 'Pe': results_baseline[:, 1], 'Pmfr': results_baseline[:, 2],
                         'Pfr': results_baseline[:, 3], 'Income': results_baseline[:, 4]}) # Creating pandas dataframe from numpy array
baseline.to_csv(r"Baseline.csv", index=False) # export baseline results

# WRC 1 results
wrc1 = pd.DataFrame({'Pfmax': results_wrc1[:, 0], 'Pe': results_wrc1[:, 1], 'Pmfr': results_wrc1[:, 2],
                    'Pfr': results_wrc1[:, 3], 'Income': results_wrc1[:, 4]}) # Creating pandas dataframe from numpy array
wrc1.to_csv(r"WRC1.csv", index=False) # export WRC 1 results

# WRC 2 results
wrc2 = pd.DataFrame({'Pe': results_wrc2[:, 0], 'Pmfr': results_wrc2[:, 1], 'Pfr': results_wrc2[:, 2],
                    'Income': results_wrc2[:, 3]}) # Creating pandas dataframe from numpy array
wrc2.to_csv(r"WRC2.csv", index=False) # export WRC 2 results

# Real power curve results
real_income_pc = pd.DataFrame({'Income': real_power_curve[:, 0]}) # Creating pandas dataframe from numpy array
real_income_pc.to_csv(r"Real_power_curve.csv", index=False) # export real power curve results

# Computation time
time = timerpc() - start_time # time in seconds
time/60 # time in minutes

28.724526980583335